In [1]:
#@title ##Import libraries!

!pip -q install pyngrok
!pip -q install streamlit

from pyngrok import ngrok

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import model_selection
from sklearn.model_selection import train_test_split

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Embedding, Dense, SimpleRNN, LSTM
from keras.wrappers.scikit_learn import KerasClassifier
import keras.optimizers as optimizers
from keras.callbacks import ModelCheckpoint
monitor = ModelCheckpoint('./model.hdf5', 
                          monitor='val_accuracy', 
                          verbose=0, 
                          save_best_only=True, 
                          save_weights_only=False, 
                          mode='auto', 
                          save_freq='epoch')

## Set random seed for reproducible results
RAND_SEED = 12
np.random.seed(RAND_SEED)
tf.random.set_seed(RAND_SEED)

## Utils function to combine 23 chunks from the same patient into one big chunk
def prepare_data(eeg_df):
  file_names = eeg_df['Unnamed: 0'].tolist()

  subject_ids = []
  chunk_ids = []
  for fn in file_names:
    subject_ids.append(fn.split('.')[-1])
    chunk_ids.append(fn.split('.')[0])
  subject_ids = list(set(subject_ids))
  assert len(subject_ids) == 500

  sub2ind = {}
  for ind, sub in enumerate(subject_ids):
    sub2ind[sub] = ind

  eeg_combined = np.zeros((500, int(178*23)))
  labels_combined = np.zeros(500)
  labels_chunks = np.zeros((500, 23))
  labels_dict = {}
  for i in range(len(eeg_df)):
    fn = eeg_df.iloc[i]['Unnamed: 0']
    subject_id = fn.split('.')[-1]
    subject_ind = sub2ind[subject_id]

    chunk_id = int(fn.split('.')[0].split('X')[-1])
    start_idx = (chunk_id - 1) * 178
    end_idx = start_idx + 178
    eeg_combined[subject_ind, start_idx:end_idx] = eeg_df.iloc[i].values[1:-1]

    if subject_id not in labels_dict:
      labels_dict[subject_id] = []
    labels_dict[subject_id].append(eeg_df.iloc[i].values[-1])

  for sub_id, labels in labels_dict.items():
    sub_ind = sub2ind[sub_id]
    is_seizure = int(np.any(np.array(labels) == 1))
    labels_combined[sub_ind] = is_seizure
    labels = np.array(labels)
    labels = np.where(labels>1, 0, labels)
    labels_chunks[sub_ind,:] = labels

  return eeg_combined, labels_combined, labels_chunks

     |████████████████████████████████| 747kB 5.8MB/s 
     |████████████████████████████████| 7.0MB 5.1MB/s 
     |████████████████████████████████| 81kB 5.1MB/s 
     |████████████████████████████████| 112kB 50.2MB/s 
     |████████████████████████████████| 163kB 41.5MB/s 
     |████████████████████████████████| 4.6MB 50.0MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 122kB 48.8MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.


In [2]:
#@title ## Prepare our data set!

#eeg = pd.read_csv(uci_epilepsy)
eeg = pd.read_csv('data.csv')
x, y, y_time_steps = prepare_data(eeg)

# reshape x into (number_of_samples, number_of_time_steps, feature dimension)
x = x.reshape(-1, 23, 178).astype(np.float32) 
np.save('sample', x[30])

# reshape y into (num_of_samples, 1)
y = y.astype(int).reshape(-1,1) 

print('Input x shape: ', x.shape)
print('Label y shape: ', y.shape)

Input x shape:  (500, 23, 178)
Label y shape:  (500, 1)


In [3]:
#@title ## Train our model!

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2020)

# Build a LSTM
model = Sequential()
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(Dense(1, activation='sigmoid'))

# Compile the LSTM
model.compile(loss='binary_crossentropy',
              optimizer = 'adam', 
              metrics = ['accuracy'])

# Train the LSTM
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, callbacks=[monitor])

# Predict on test data
predictions = model.predict(x_test)
predictions = predictions > 0.5

### END CODE
print('Test accuracy: ', accuracy_score(y_test, predictions[:, -1, :]))
model.save('model.h5')

Epoch 1/20
13/13 [==============================] - 13s 233ms/step - loss: 0.6706 - accuracy: 0.6813 - val_loss: 0.5525 - val_accuracy: 0.8100
Epoch 2/20
13/13 [==============================] - 1s 58ms/step - loss: 0.5288 - accuracy: 0.7923 - val_loss: 0.4786 - val_accuracy: 0.8100
Epoch 3/20
13/13 [==============================] - 1s 55ms/step - loss: 0.4533 - accuracy: 0.8012 - val_loss: 0.4093 - val_accuracy: 0.8430
Epoch 4/20
13/13 [==============================] - 1s 55ms/step - loss: 0.3662 - accuracy: 0.8837 - val_loss: 0.3867 - val_accuracy: 0.8674
Epoch 5/20
13/13 [==============================] - 1s 59ms/step - loss: 0.3044 - accuracy: 0.9130 - val_loss: 0.3545 - val_accuracy: 0.8817
Epoch 6/20
13/13 [==============================] - 1s 54ms/step - loss: 0.2482 - accuracy: 0.9323 - val_loss: 0.3827 - val_accuracy: 0.8630
Epoch 7/20
13/13 [==============================] - 1s 56ms/step - loss: 0.2217 - accuracy: 0.9373 - val_loss: 0.4106 - val_accuracy: 0.8600
Epoch 8/20


In [4]:
#@title ## Build our web application!

%%writefile app.py
import streamlit as st
from joblib import dump, load

import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

model = keras.models.load_model('model.h5')

st.title('Epilepsy Detector') 
uploaded_file = st.file_uploader('Upload Data')
if uploaded_file is not None:
  data = np.load(uploaded_file).reshape(1, 23, 178)
  pred = model.predict(data)
  confidence = pred[:, -1, :][0][0]
  final_pred = pred > 0.5
  final_pred = final_pred[:, -1, :][0][0]
  if final_pred == 0:
    st.write(f'The model is {round(100 - confidence, 2)}% confident that this patient DOES NOT have epilepsy.') 
  else:
    st.write(f'The model is {round(confidence * 100, 2)}% confident that this patient DOES HAVE have epilepsy.') 
  st.line_chart(pd.DataFrame(pred.reshape(-1, 1)))


Writing app.py


In [7]:
#@title ## Run our web application!

public_url = ngrok.connect(port='80')
print(public_url)
!streamlit run app.py >/dev/null

NgrokTunnel: "http://6a68db384611.ngrok.io" -> "http://localhost:80"
2021-03-27 20:02:50.362988: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-27 20:02:51.827261: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-27 20:02:51.828214: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-27 20:02:51.843579: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-03-27 20:02:51.843635: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (8bef5eb6c454): /proc/driver/nvidia/version does not exist
2021-03-27 20:02:51.844369: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [6]:
#@title If you get an error above, run this to reinitialize Streamlit, then try again!
%%writefile ~/.streamlit/config.toml
[server]
port = 80

Writing /root/.streamlit/config.toml
